In [1]:
import pandas as pd
import numpy as np
from sklearn.utils import resample

In [2]:
# jiji = pd.read_csv("jiji_clean.csv")
autorec = pd.read_csv("clean_data/autorec_clean.csv")
bef = pd.read_csv("clean_data/beforward_clean.csv")

In [3]:
print("Loaded:")
# print("Jiji:", jiji.shape)
print("Autorec:", autorec.shape)
print("BeForward:", bef.shape)

Loaded:
Autorec: (50, 9)
BeForward: (2446, 16)


In [4]:
autorec["source"] = "Autorec"
bef["source"] = "BeForward"

In [5]:
master_columns = [
    "ref_no",
    "title",
    "make",
    "model",
    "year",
    "mileage",
    "engine_cc",
    "transmission",
    "fuel",
    "seats",
    "location",
    "price_usd",
    "total_price_usd",
    "price_ugx",
    "total_price_ugx",
    "url",
    "source"
]

In [ ]:
# jiji_merged = pd.DataFrame({
#     "source": jiji["source"],
#     "make": jiji["make"],
#     "model": jiji["model"],
#     "year": jiji["year"],
#     "mileage": jiji["mileage"],
#     "engine_cc": None,
#     "transmission": jiji["transmission"],
#     "fuel": jiji["fuel"],
#     "location": jiji.get("location", None),
#     "price_ugx": jiji["price"],
#     "url": jiji["url"]
# })

In [6]:
bef_mapped = bef.copy()
bef_mapped["source"] = "BeForward"

# Make sure all master columns exist
for col in master_columns:
    if col not in bef_mapped.columns:
        bef_mapped[col] = None

In [7]:
autorec_mapped = pd.DataFrame(columns=master_columns)

autorec_mapped["title"] = autorec["title"]
autorec_mapped["make"] = autorec["make"]
autorec_mapped["model"] = autorec["model"]
autorec_mapped["year"] = autorec["year"]
autorec_mapped["mileage"] = autorec["mileage"]
autorec_mapped["engine_cc"] = autorec["engine_cc"]

# Prices
autorec_mapped["price_usd"] = autorec["price_usd"]
autorec_mapped["price_ugx"] = autorec["price_ugx"]

# Missing columns for autorec
autorec_mapped["ref_no"] = None
autorec_mapped["transmission"] = autorec.get("transmission", None)
autorec_mapped["fuel"] = None
autorec_mapped["seats"] = None
autorec_mapped["location"] = None

# No total prices in autorec
autorec_mapped["total_price_usd"] = None
autorec_mapped["total_price_ugx"] = None

autorec_mapped["url"] = autorec["url"]
autorec_mapped["source"] = "Autorec"

In [8]:
oversample_factor = max(1, int(len(bef_mapped) / len(autorec_mapped)))

autorec_oversampled = resample(
    autorec_mapped,
    replace=True,
    n_samples=len(autorec_mapped) * oversample_factor,
    random_state=42
)

print("Autorec oversampled size:", autorec_oversampled.shape)

Autorec oversampled size: (2400, 17)


In [11]:
merged = pd.concat([bef_mapped, autorec_oversampled], ignore_index=True)

/var/folders/k7/ggnq5s_d0tg4z20frzwdzy_r0000gn/T/ipykernel_55642/2031093191.py:1: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  merged = pd.concat([bef_mapped, autorec_oversampled], ignore_index=True)


In [ ]:
# merged = pd.concat([jiji_merged, autorec_merged, bef_merged], ignore_index=True)
# merged = pd.concat([autorec_merged, bef_merged], ignore_index=True)
# print("\nMerged shape before cleaning:", merged.shape)

NameError: name 'autorec_merged' is not defined

In [12]:
merged = merged[master_columns]
print("Merged dataset shape:", merged.shape)

Merged dataset shape: (4846, 17)


In [13]:
merged.to_csv("clean_data/cars_merged.csv", index=False)

print("\nSaved unified dataset → cars_merged.csv")
merged.head(10)


Saved unified dataset → cars_merged.csv


,ref_no,title,make,model,year,mileage,engine_cc,transmission,fuel,seats,location,price_usd,total_price_usd,price_ugx,total_price_ugx,url,source
0,BY759022,2006 HONDA ACTY TRUCK,Honda,Acty,2006,111250.0,650.0,MT,Petrol,2,Location,2370,39730.0,8532000,143028000.0,/honda/acty-truck/by759022/id/11910020/,BeForward
1,BY759024,2013 MITSUBISHI CANTER,Mitsubishi,Canter,2013,312401.0,2990.0,Semi AT,Diesel,3,Location,3920,77470.0,14112000,278892000.0,/mitsubishi/canter/by759024/id/11910065/,BeForward
2,CA462307,2014 TOYOTA HIACE VAN\n ...,Toyota,Hiace,2014,161352.0,2980.0,AT,Diesel,3,Location,5960,84920.0,21456000,305712000.0,/toyota/hiace-van/ca462307/id/12565207/,BeForward
3,CA740447,2011 TOYOTA WISH\n ...,Toyota,Wish,2011,154001.0,1790.0,AT,Petrol,7,Location,2430,45440.0,8748000,163584000.0,/toyota/wish/ca740447/id/12831915/,BeForward
4,CB026605,2017 HONDA FIT HYBRID\n ...,Honda,Fit,2017,111072.0,1490.0,AT,Hybrid(Petrol),5,Location,4790,66330.0,17244000,238788000.0,/honda/fit-hybrid/cb026605/id/13106276/,BeForward
5,CB205955,2022 MAZDA MAZDA2\n ...,Mazda,Mazda2,2022,74400.0,1299.0,AT,Petrol,5,Location,12190,135390.0,43884000,487404000.0,/mazda/mazda2/cb205955/id/13263374/,BeForward
6,CB205954,2016 TOYOTA HILUX\n ...,Toyota,Hilux,2016,13000.0,2393.0,MT,Diesel,4,Location,12920,149940.0,46512000,539784000.0,/toyota/hilux/cb205954/id/13263373/,BeForward
7,CB205953,2019 TOYOTA HILUX\n ...,Toyota,Hilux,2019,48000.0,2393.0,AT,Diesel,5,Location,19670,217440.0,70812000,782784000.0,/toyota/hilux/cb205953/id/13263372/,BeForward
8,CB205952,2017 FORD RANGER\n ...,Ford,Ranger,2017,50000.0,2198.0,AT,Diesel,5,Location,15140,172540.0,54504000,621144000.0,/ford/ranger/cb205952/id/13263371/,BeForward
9,CB205951,2012 FORD RANGER\n ...,Ford,Ranger,2012,65000.0,2198.0,AT,Diesel,5,Location,11560,136740.0,41616000,492264000.0,/ford/ranger/cb205951/id/13263370/,BeForward
